In [ ]:
:tags: [thebe-init]
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

In [ ]:
# Vuoksi Macro Lab: Two-Period Consumption (Euler Equation Edition)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True

# --- Two-Period Consumption with Euler Equation and Utility Map ---
def two_period_upgraded(y1=15.0, y2=98.0, r=0.05, beta=1.0, view='Utility Map'):
    # Intertemporal budget constraint
    PV_income = y1 + y2 / (1 + r)
    c1_star = PV_income / (1 + beta)
    c2_star = beta * (1 + r) * c1_star

    # Euler line
    c1_vals = np.linspace(0.1, PV_income, 200)
    euler_line = beta * (1 + r) * c1_vals

    # Budget line
    c2_budget = (PV_income - c1_vals) * (1 + r)

    # Plot
    plt.figure(figsize=(8, 6))

    if view == 'Utility Map':
        # Plot indifference curves (log utility)
        for u_level in [np.log(c1_star) + np.log(c2_star) - i for i in [0.8, 0.4, 0, -0.4]]:
            indiff_c2 = np.exp(u_level - np.log(c1_vals))
            plt.plot(c1_vals, indiff_c2, linestyle='--', color='gray', alpha=0.6)

    plt.plot(c1_vals, c2_budget, label='Budget Line', color='black')
    plt.plot(c1_vals, euler_line, label='Euler Line', color='blue')
    plt.scatter(c1_star, c2_star, color='red', s=100, label='Optimal (c₁*, c₂*)')

    plt.xlabel("Consumption in Period 1 (c₁)")
    plt.ylabel("Consumption in Period 2 (c₂)")
    plt.title("Two-Period Consumption: Budget, Euler & Utility")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    s = y1 - c1_star
    print(f"PV Income = {PV_income:.2f}")
    print(f"Optimal c₁*: {c1_star:.2f}, c₂*: {c2_star:.2f}")
    print(f"Savings in Period 1: {s:.2f} → {'Saving' if s > 0 else 'Borrowing'}")

interact(
    two_period_upgraded,
    y1=FloatSlider(value=15.0, min=0.0, max=50.0, step=1.0, description='Income y₁'),
    y2=FloatSlider(value=98.0, min=0.0, max=200.0, step=5.0, description='Income y₂'),
    r=FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='Interest Rate r'),
    beta=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Discount Factor β'),
    view=Dropdown(options=['Utility Map', 'Clean View'], value='Utility Map', description='View Mode'),
);


interactive(children=(FloatSlider(value=15.0, description='Income y₁', max=50.0, step=1.0), FloatSlider(value=…